In [1]:
# !pip install folium

In [2]:
import pandas as pd 
import folium

In [3]:

drivers = pd.read_csv("https://raw.githubusercontent.com/terrydiko/INST447Project/main/Drivers.csv", encoding='utf-8', dtype="str")
drivers

,report_number,local_case_number,agency_name,acrs_report_type,crash_date_time,route_type,road_name,cross_street_type,cross_street_name,off_road_description,...,speed_limit,driverless_vehicle,parked_vehicle,vehicle_year,vehicle_make,vehicle_model,equipment_problems,latitude,longitude,geolocation
0,MCP3239001B,210015908,Montgomery County Police,Injury Crash,2021-04-28T21:09:00.000,NaN,NaN,NaN,NaN,IN PARKING LOT IN FRONT OF 13232 STRAVINSKY TE...,...,15,No,No,2006,CADILLAC,4D,NO MISUSE,39.04605063,-76.99075732,", \n(39.04605063, -76.99075732)"
1,MCP3290000G,210015906,Montgomery County Police,Property Damage Crash,2021-04-28T21:01:00.000,County,GALWAY DR,County,GALWAY PL,NaN,...,40,No,No,2006,CADILLAC,STS,UNKNOWN,39.059336,-76.944363,", \n(39.059336, -76.944363)"
2,MCP3283000M,210015903,Montgomery County Police,Property Damage Crash,2021-04-28T20:37:00.000,County,GLEN RD,County,GLEN CT,NaN,...,30,No,No,2018,HONDA,ODYSSEY,NO MISUSE,39.0471,-77.225175,", \n(39.0471, -77.225175)"
3,MCP2892009B,210015889,Montgomery County Police,Property Damage Crash,2021-04-28T17:58:00.000,Maryland (State),GEORGIA AVE,Maryland (State),FOREST GLEN RD,NaN,...,35,No,No,2013,NEW,TBU,NO MISUSE,39.01532,-77.042505,", \n(39.01532, -77.042505)"
4,MCP2892009B,210015889,Montgomery County Police,Property Damage Crash,2021-04-28T17:58:00.000,Maryland (State),GEORGIA AVE,Maryland (State),FOREST GLEN RD,NaN,...,35,No,No,0,UNKNOWN,UNKNOWN,NaN,39.01532,-77.042505,", \n(39.01532, -77.042505)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123287,MCP2783000G,15000025,Montgomery County Police,Injury Crash,2015-01-01T01:14:00.000,Maryland (State),NORBECK RD,County,BAUER DR,NaN,...,40,No,No,2002,TOYOTA,CAMRY,NaN,39.09621667,-77.10980333,", \n(39.09621667, -77.10980333)"
123288,MCP2686000D,15000017,Montgomery County Police,Property Damage Crash,2015-01-01T01:10:00.000,Ramp,RAMP 5 FR RAMP 8 TO FATHER HURLEY BLVD,County,FATHER HURLEY BLVD,NaN,...,40,No,No,2007,JEEP,COMPASS,NO MISUSE,39.19661,-77.26554667,", \n(39.19661, -77.26554667)"
123289,MCP2686000D,15000017,Montgomery County Police,Property Damage Crash,2015-01-01T01:10:00.000,Ramp,RAMP 5 FR RAMP 8 TO FATHER HURLEY BLVD,County,FATHER HURLEY BLVD,NaN,...,40,No,No,2010,FORD,FOCUS,NO MISUSE,39.19661,-77.26554667,", \n(39.19661, -77.26554667)"
123290,MCP11520005,15000008,Montgomery County Police,Property Damage Crash,2015-01-01T00:30:00.000,Maryland (State),ROCKVILLE PIKE,County,GROSVENOR LA,NaN,...,35,No,No,2008,RANGE ROVER,HSC,NaN,39.02200107,-77.10267305,", \n(39.02200107, -77.10267305)"


In [4]:
incidents = pd.read_csv("https://raw.githubusercontent.com/terrydiko/INST447Project/main/Incidents.csv", encoding= 'utf-8', dtype="str")
incidents

,report_number,local_case_number,agency_name,acrs_report_type,crash_date_time,hit_run,route_type,mile_point,mile_point_direction,lane_direction,...,fixed_oject_struck,junction,intersection_type,intersection_area,road_alignment,road_condition,road_division,latitude,longitude,geolocation
0,MCP3239001B,210015908,Montgomery County Police,Injury Crash,2021-04-28T21:09:00.000,No,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.04605063,-76.99075732,", \n(39.04605063, -76.99075732)"
1,MCP3290000G,210015906,Montgomery County Police,Property Damage Crash,2021-04-28T21:01:00.000,Yes,County,0.2,North,South,...,NaN,NON INTERSECTION,NaN,NaN,STRAIGHT,NO DEFECTS,"TWO-WAY, NOT DIVIDED",39.059336,-76.944363,", \n(39.059336, -76.944363)"
2,MCP3283000M,210015903,Montgomery County Police,Property Damage Crash,2021-04-28T20:37:00.000,No,County,0.69,West,East,...,OTHER POLE,NON INTERSECTION,NaN,NaN,CURVE LEFT,NO DEFECTS,"TWO-WAY, NOT DIVIDED",39.0471,-77.225175,", \n(39.0471, -77.225175)"
3,MCP2892009B,210015889,Montgomery County Police,Property Damage Crash,2021-04-28T17:58:00.000,Yes,Maryland (State),1.61,North,North,...,NaN,NaN,NaN,NaN,STRAIGHT,NO DEFECTS,"TWO-WAY, NOT DIVIDED",39.01532,-77.042505,", \n(39.01532, -77.042505)"
4,MCP2899004S,210015885,Montgomery County Police,Property Damage Crash,2021-04-28T17:34:00.000,Yes,Maryland (State),4.54,North,South,...,GUARDRAIL OR BARRIER,NaN,NaN,NaN,CURVE LEFT,NO DEFECTS,"TWO-WAY, DIVIDED, POSITIVE MEDIAN BARRIER",39.139095,-77.26846667,", \n(39.139095, -77.26846667)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69553,MCP23000002,15000197,Montgomery County Police,Property Damage Crash,2015-01-01T02:00:00.000,No,County,1.81,North,North,...,NaN,NON INTERSECTION,NaN,NaN,STRAIGHT,NO DEFECTS,"TWO-WAY, NOT DIVIDED",39.148678,-77.093848,"(39.148678, -77.093848)"
69554,MCP2674000D,15000016,Montgomery County Police,Property Damage Crash,2015-01-01T01:26:00.000,No,County,0.76,North,West,...,NaN,INTERSECTION,FOUR-WAY INTERSECTION,INTERSECTION,STRAIGHT,NO DEFECTS,"TWO-WAY, DIVIDED, POSITIVE MEDIAN BARRIER",39.158059,-77.204349,"(39.158059, -77.204349)"
69555,MCP2783000G,15000025,Montgomery County Police,Injury Crash,2015-01-01T01:14:00.000,No,Maryland (State),25.21,East,East,...,NaN,INTERSECTION,FOUR-WAY INTERSECTION,NaN,STRAIGHT,NO DEFECTS,"TWO-WAY, DIVIDED, POSITIVE MEDIAN BARRIER",39.09621667,-77.10980333,"(39.09621667, -77.10980333)"
69556,MCP2686000D,15000017,Montgomery County Police,Property Damage Crash,2015-01-01T01:10:00.000,No,Ramp,0.15,South,South,...,NaN,INTERCHANGE RELATED,T-INTERSECTION,ON RAMP EXIT AREA,CURVE LEFT,NO DEFECTS,ONE-WAY TRAFFICWAY,39.19661,-77.26554667,"(39.19661, -77.26554667)"


In [7]:
merge_data = incidents.merge(drivers, on='report_number', how='inner')
merge_data[merge_data.vehicle_make == ""]
merge_data.query("vehicle_name != 'UNKNOWN'")

# Project Focus/Goal
# What factors have the biggest influence in car accidents?
# Factors: Year of Vehicle, Time of Accident, Weather, Number of Lanes, Speed Limit


,report_number,local_case_number_x,agency_name_x,acrs_report_type_x,crash_date_time_x,hit_run,route_type_x,mile_point,mile_point_direction,lane_direction,...,speed_limit,driverless_vehicle,parked_vehicle,vehicle_year,vehicle_make,vehicle_model,equipment_problems,latitude_y,longitude_y,geolocation_y
0,MCP3239001B,210015908,Montgomery County Police,Injury Crash,2021-04-28T21:09:00.000,No,NaN,NaN,NaN,NaN,...,15,No,No,2006,CADILLAC,4D,NO MISUSE,39.04605063,-76.99075732,", \n(39.04605063, -76.99075732)"
1,MCP3290000G,210015906,Montgomery County Police,Property Damage Crash,2021-04-28T21:01:00.000,Yes,County,0.2,North,South,...,40,No,No,2006,CADILLAC,STS,UNKNOWN,39.059336,-76.944363,", \n(39.059336, -76.944363)"
2,MCP3283000M,210015903,Montgomery County Police,Property Damage Crash,2021-04-28T20:37:00.000,No,County,0.69,West,East,...,30,No,No,2018,HONDA,ODYSSEY,NO MISUSE,39.0471,-77.225175,", \n(39.0471, -77.225175)"
3,MCP2892009B,210015889,Montgomery County Police,Property Damage Crash,2021-04-28T17:58:00.000,Yes,Maryland (State),1.61,North,North,...,35,No,No,2013,NEW,TBU,NO MISUSE,39.01532,-77.042505,", \n(39.01532, -77.042505)"
4,MCP2892009B,210015889,Montgomery County Police,Property Damage Crash,2021-04-28T17:58:00.000,Yes,Maryland (State),1.61,North,North,...,35,No,No,0,UNKNOWN,UNKNOWN,NaN,39.01532,-77.042505,", \n(39.01532, -77.042505)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123287,MCP2783000G,15000025,Montgomery County Police,Injury Crash,2015-01-01T01:14:00.000,No,Maryland (State),25.21,East,East,...,40,No,No,2002,TOYOTA,CAMRY,NaN,39.09621667,-77.10980333,", \n(39.09621667, -77.10980333)"
123288,MCP2686000D,15000017,Montgomery County Police,Property Damage Crash,2015-01-01T01:10:00.000,No,Ramp,0.15,South,South,...,40,No,No,2007,JEEP,COMPASS,NO MISUSE,39.19661,-77.26554667,", \n(39.19661, -77.26554667)"
123289,MCP2686000D,15000017,Montgomery County Police,Property Damage Crash,2015-01-01T01:10:00.000,No,Ramp,0.15,South,South,...,40,No,No,2010,FORD,FOCUS,NO MISUSE,39.19661,-77.26554667,", \n(39.19661, -77.26554667)"
123290,MCP11520005,15000008,Montgomery County Police,Property Damage Crash,2015-01-01T00:30:00.000,Yes,Maryland (State),4.38,North,North,...,35,No,No,2008,RANGE ROVER,HSC,NaN,39.02200107,-77.10267305,", \n(39.02200107, -77.10267305)"


In [7]:
# spped


,X,Y,OBJECTID_x,CRIMEID,CCN,REPORTDATE,ROUTEID,MEASURE,OFFSET,STREETSEGID,FROMDATE,TODATE,MARID,ADDRESS_x,OBJECTID_y,ID,ENFORCEMENT,ADDRESS_y
0,-76.9780827,38.93754653,201587003,26624955,12051854,2012/04/16 05:00:00+00,12081442,2913.3,0.05,5751,2012/04/16 04:00:00+00,1970/01/01 00:00:00+00,806707,3900 BLK SOUTH DAKOTA AVE NE,109,51,SPEED SAFETY CAMERA,3900 blk South Dakota Ave NE se/b
1,-76.9780827,38.93754653,201587003,26624955,12051854,2012/04/16 05:00:00+00,12081442,2913.3,0.05,5751,2012/04/16 04:00:00+00,1970/01/01 00:00:00+00,806707,3900 BLK SOUTH DAKOTA AVE NE,110,54,SPEED SAFETY CAMERA,3900 blk South Dakota Ave NE nw/b
2,-76.9780827,38.93754653,201656001,26596685,8122951,2008/08/30 05:00:00+00,12081442,2913.3,0.05,5751,2008/08/30 04:00:00+00,1970/01/01 00:00:00+00,806707,3900 BLOCK SOUTH DAKOTA AVE. NE,109,51,SPEED SAFETY CAMERA,3900 blk South Dakota Ave NE se/b
3,-76.9780827,38.93754653,201656001,26596685,8122951,2008/08/30 05:00:00+00,12081442,2913.3,0.05,5751,2008/08/30 04:00:00+00,1970/01/01 00:00:00+00,806707,3900 BLOCK SOUTH DAKOTA AVE. NE,110,54,SPEED SAFETY CAMERA,3900 blk South Dakota Ave NE nw/b
4,-76.9780827,38.93754653,201697743,24172560,12119555,2012/08/24 05:00:00+00,12081442,2913.3,0.05,5751,2012/08/24 04:00:00+00,1970/01/01 00:00:00+00,806707,3900 BLOCK OF SOUTH DAKOTA AVE NE,109,51,SPEED SAFETY CAMERA,3900 blk South Dakota Ave NE se/b
5,-76.9780827,38.93754653,201697743,24172560,12119555,2012/08/24 05:00:00+00,12081442,2913.3,0.05,5751,2012/08/24 04:00:00+00,1970/01/01 00:00:00+00,806707,3900 BLOCK OF SOUTH DAKOTA AVE NE,110,54,SPEED SAFETY CAMERA,3900 blk South Dakota Ave NE nw/b
6,-77.00891898,38.90740626,201587068,26625037,12053630,2012/04/21 05:00:00+00,12064672,12.96,0.06,13000,2012/04/20 04:00:00+00,1970/01/01 00:00:00+00,913908,NEW YORK AVE NE and NORTH CAPITOL ST NE,53,41,RED LIGHT SAFETY CAMERA,New York Ave sw/b @ N Capitol St NE
7,-77.00891898,38.90740626,201592003,25113140,14037439,2014/03/19 05:00:00+00,12064672,12.96,0.06,13000,2014/03/19 04:00:00+00,1970/01/01 00:00:00+00,913908,NEW YORK AVE NE and NORTH CAPITOL ST,53,41,RED LIGHT SAFETY CAMERA,New York Ave sw/b @ N Capitol St NE
8,-77.00905526,38.90740678,201594975,24231869,12136609,2012/09/27 16:05:00+00,15065322,1170.48,11.79,13000,2012/09/27 04:00:00+00,1970/01/01 00:00:00+00,913908,NEW YORK AVE. & NORTH CAPITOL ST. NE.,53,41,RED LIGHT SAFETY CAMERA,New York Ave sw/b @ N Capitol St NE
9,-77.00891898,38.90740626,201609128,25055888,14007850,2014/01/17 05:00:00+00,12064672,12.96,0.06,13000,2014/01/17 05:00:00+00,1970/01/01 00:00:00+00,913908,NEW YORK AVE NE and NORTH CAPITOL ST NE,53,41,RED LIGHT SAFETY CAMERA,New York Ave sw/b @ N Capitol St NE


In [8]:
x_y = []
for index in range(0, len(merge_crash['Y'])):
    x_y.append([float(merge_crash['Y'][index]), float(merge_crash['X'][index])])
x_y[:10]

[[38.93754653, -76.9780827],
 [38.93754653, -76.9780827],
 [38.93754653, -76.9780827],
 [38.93754653, -76.9780827],
 [38.93754653, -76.9780827],
 [38.93754653, -76.9780827],
 [38.90740626, -77.00891898],
 [38.90740626, -77.00891898],
 [38.90740678, -77.00905526],
 [38.90740626, -77.00891898]]

In [9]:
m = folium.Map(location=[38.86948278, -76.96897301], default_zoom_start=200)

for xy in x_y:
    folium.Marker(location=xy, popup='You found a crash').add_to(m)

m

In [10]:
enforment = merge_crash[['ENFORCEMENT', 'TODATE']].groupby('ENFORCEMENT').agg('count').reset_index()
enforment.columns = ['ENFORCEMENT', 'Number-of-Accident']
enforment

,ENFORCEMENT,Number-of-Accident
0,OVERSIZED SAFETY CAMERA,2
1,RED LIGHT SAFETY CAMERA,1027
2,SPEED SAFETY CAMERA,255
3,STOP SIGN SAFETY CAMERA,10
